<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/BulkLoadingAndFiltering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bulk loading, pre-processing, pre-filtering of data

NOTE: copied from 'BulkVisualisations_1.ipynb', which wasn't completed, but that file did a lot of getting the data into a usable shape.

# Setup env


In [1]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 43
-rw------- 1 root root 22877 Jul  2 21:06 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  7276 Jul  6 14:14 stats_n_features.py
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [4]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

     |████████████████████████████████| 61kB 1.9MB/s 


In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

>> LOADING custom module, when: 2020-07-31 18:13:33.316708, module name: data_prep


In [7]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function in data_prep.py called and worked! when: 2020-07-31 18:13:33.388362,  param:bling


In [9]:
import stats_n_features as sf

>> LOADING custom module, when: 2020-07-31 18:13:33.694287, module name: stats_n_features


In [10]:
sf.test_function_call('hello')

Test function in stats_n_features called and worked! when: 2020-07-31 18:13:33.733744,  param:hello


## Pandas display options

In [11]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Bulk Load Data

There's a csv of meta info on all the various MIDI files in the data bundle. This part of the code plays around with that, loading/ filtering/ tweaking some columns, etc.

Not convinced there's much benefit to this approach every time, probably worth doing a bit of manipulation and saving out the results into a DataFrame usable later.

### Setup

This does some tweaking of the info.csv, sets the types, then filters the list to those in the 'eval_session' section, then adds new column prefixing filenames with full paths so they can be easily loaded.

In [12]:
metafile = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/info.csv'

meta_df = pd.read_csv(metafile, dtype = {"drummer": "string", 
                                         "session" : "string", 
                                         "id": "string", 
                                         "style": "string", 
                                         "beat_type": "string", 
                                         "time_signature" : "string",
                                         "midi_filename" : "string",
                                         "audio_filename" : "string",
                                         "split" : "string"})

# show count of test/ train/ validation split specified in file...
meta_df.groupby(['split', 'drummer']).size()


split       drummer  
test        drummer1      48
            drummer3       8
            drummer5      13
            drummer7      40
            drummer8      17
            drummer9       3
train       drummer1     386
            drummer10     10
            drummer2      17
            drummer3      83
            drummer4       7
            drummer5      36
            drummer6      11
            drummer7     263
            drummer8      58
            drummer9      26
validation  drummer1      60
            drummer2       1
            drummer3       7
            drummer4       2
            drummer5       5
            drummer6       4
            drummer7      39
            drummer8       5
            drummer9       1
dtype: int64

### Filter to just the 'eval_session' songs

In [13]:
# pull out the data for just 'eval_session' ..
eval_df = meta_df[meta_df['session'].str.contains('eval_session')].copy()
eval_df.info()

# display(eval_df)
eval_df['style'].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 1033
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   drummer         40 non-null     string 
 1   session         40 non-null     string 
 2   id              40 non-null     string 
 3   style           40 non-null     string 
 4   bpm             40 non-null     int64  
 5   beat_type       40 non-null     string 
 6   time_signature  40 non-null     string 
 7   midi_filename   40 non-null     string 
 8   audio_filename  40 non-null     string 
 9   duration        40 non-null     float64
 10  split           40 non-null     string 
dtypes: float64(1), int64(1), string(9)
memory usage: 3.8 KB


<StringArray>
[  'funk/groove1',  'soul/groove10',   'funk/groove2',   'soul/groove3',
   'soul/groove4',   'funk/groove5', 'hiphop/groove6',    'pop/groove7',
   'rock/groove8',   'soul/groove9']
Length: 10, dtype: string

In [14]:
eval_df.drop(columns=['audio_filename', 'duration', 'beat_type'], inplace=True)

eval_df.groupby('drummer', as_index=False).count()
print('list of unique drummer IDs: {}'.format(eval_df['drummer'].unique()))

,drummer,session,id,style,bpm,time_signature,midi_filename,split
0,drummer1,10,10,10,10,10,10,10
1,drummer5,10,10,10,10,10,10,10
2,drummer7,10,10,10,10,10,10,10
3,drummer8,10,10,10,10,10,10,10


list of unique drummer IDs: <StringArray>
['drummer1', 'drummer5', 'drummer7', 'drummer8']
Length: 4, dtype: string


In [36]:
file_prefix = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/'

eval_df['long_midi_filename'] = eval_df['midi_filename'].apply(lambda x: "{}{}".format(file_prefix, x))

# eval_df.head(3)
eval_df.head(10)
eval_df.info()

,drummer,session,id,style,bpm,time_signature,midi_filename,split,long_midi_filename
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,4-4,drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid
613,drummer5,drummer5/eval_session,drummer5/eval_session/1,funk/groove1,138,4-4,drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid
616,drummer5,drummer5/eval_session,drummer5/eval_session/3,soul/groove3,86,4-4,drummer5/eval_session/3_soul-groove3_86_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/3_soul-groove3_86_beat_4-4.mid
617,drummer5,drummer5/eval_session,drummer5/eval_session/4,soul/groove4,80,4-4,drummer5/eval_session/4_soul-groove4_80_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/4_soul-groove4_80_beat_4-4.mid
619,drummer5,drummer5/eval_session,drummer5/eval_session/6,hiphop/groove6,87,4-4,drummer5/eval_session/6_hiphop-groove6_87_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/6_hiphop-groove6_87_beat_4-4.mid
621,drummer5,drummer5/eval_session,drummer5/eval_session/8,rock/groove8,65,4-4,drummer5/eval_session/8_rock-groove8_65_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/8_rock-groove8_65_beat_4-4.mid


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 1032
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   drummer             20 non-null     string
 1   session             20 non-null     string
 2   id                  20 non-null     string
 3   style               20 non-null     string
 4   bpm                 20 non-null     int64 
 5   time_signature      20 non-null     string
 6   midi_filename       20 non-null     string
 7   split               20 non-null     string
 8   long_midi_filename  20 non-null     object
dtypes: int64(1), object(1), string(7)
memory usage: 1.6+ KB


In [16]:
# this is the data I need to save
display(eval_df)

,drummer,session,id,style,bpm,time_signature,midi_filename,split,long_midi_filename
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,4-4,drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,4-4,drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,4-4,drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid
5,drummer1,drummer1/eval_session,drummer1/eval_session/5,funk/groove5,84,4-4,drummer1/eval_session/5_funk-groove5_84_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/5_funk-groove5_84_beat_4-4.mid
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid
7,drummer1,drummer1/eval_session,drummer1/eval_session/7,pop/groove7,138,4-4,drummer1/eval_session/7_pop-groove7_138_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/7_pop-groove7_138_beat_4-4.mid
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid
9,drummer1,drummer1/eval_session,drummer1/eval_session/9,soul/groove9,105,4-4,drummer1/eval_session/9_soul-groove9_105_beat_4-4.mid,test,/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/9_soul-groove9_105_beat_4-4.mid


## Store/ write the prepared csv

In [37]:
out_file = '/content/drive/My Drive/LYIT/Dissertation/data/eval_data.csv'
eval_df.to_csv(out_file)

## Filter song styles

In [17]:
# filter for just the selected song style
# NOTE: these are labels for each 'style' (1-10) the drummers
# were asked to play, after reviewing, the following were deemed
# most suitable/ usable as data for the project
song_styles = ['funk/groove1', 'soul/groove3', 'soul/groove4', 'hiphop/groove6', 'rock/groove8']
eval_df = eval_df[eval_df['style'].isin(song_styles)]
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 1032
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   drummer             20 non-null     string
 1   session             20 non-null     string
 2   id                  20 non-null     string
 3   style               20 non-null     string
 4   bpm                 20 non-null     int64 
 5   time_signature      20 non-null     string
 6   midi_filename       20 non-null     string
 7   split               20 non-null     string
 8   long_midi_filename  20 non-null     object
dtypes: int64(1), object(1), string(7)
memory usage: 1.6+ KB


In [18]:
# display(eval_df)

## Created NamedTuple container

This is just an experiment at creating convenient container to gather together all the results of various bits of processing realted to a single file, in a container so they can be conveniently accessed.

For more info, see...
* http://zetcode.com/python/namedtuple/
* https://dbader.org/blog/writing-clean-python-with-namedtuples

It's kinda like a dictionary, but not as it's not mutable, but maybe that's a good thing? Dunno ...

In [19]:

from collections import namedtuple

'''
  Collection object to hold in a single place all information
  related to a performance loaded from a MIDI file.

  drummer_id = String
  file_df = complete DataFrame of data, unfiltered
  file_wrapper = MidiFileWrapper instance
  tools = MidiTimingTools instance
  stats_df = DataFrame fleshed out with additional features/ metrics
'''
PerformanceData = namedtuple('PerformanceData' , 'drummer_id file_df file_wrapper tools stats_df')


'\n  Collection object to hold in a single place all information\n  related to a performance loaded from a MIDI file.\n\n  drummer_id = String\n  file_df = complete DataFrame of data, unfiltered\n  file_wrapper = MidiFileWrapper instance\n  tools = MidiTimingTools instance\n  stats_df = DataFrame fleshed out with additional features/ metrics\n'

## Load & process files, make uber bucket of usable data

This loads the files, purges any quantise positions (buckets) with hits per instrument > 1 in a bucket. i.e. notes happening more rapidly than we cater for, we're only handling detail to 16th notes.

In [20]:
all_drummer_data = {}

# iterate over rows with iterrows()
for index, row in eval_df.iterrows():

  # access data using column names
  next_drummer = row['drummer']
  long_name = row['long_midi_filename']
  short_name = row['midi_filename']

  # loads DataFrame (file_df), MIDI_File_Wrapper (f), and
  # the associated MidiTimingTools (mtt) objects..
  file_df, file_wrapper, mtt = dp.load_file(long_name)

  print('    > checking for errs: {}'.format(short_name))

  #### review data, see if errors to be removed...

  err_buckets = sf.get_error_buckets(file_df) # parse for problem beats
  if err_buckets.size == 0:
    print('    ...no errors to see here')
  else: # handle buckets > 1 hit for instrument
    #display(err_buckets)
    print('    __ file_df before: {}'.format(file_df.shape))
    print('    __ err_buckets removed: {}'.format(err_buckets.shape))
    file_df.drop(err_buckets.index, inplace=True) # remove errs, inplace
    print('    __ file_df after: {}'.format(file_df.shape))

  # gather together stats on cleaned up file_df
  stats_df = sf.gather_stats(file_df) # parse to gather stats

  # add tuple of data elements to dict with filename as key
  all_drummer_data[long_name] = PerformanceData(next_drummer, file_df, file_wrapper, mtt, stats_df)


FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    __notes pre-filter: [36. 37. 38. 40. 43. 44. 51. 52. 53. 55.]
    __applying filter: [44]
    __notes post filter: [36. 37. 38. 40. 43. 51. 52. 53. 55.]
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 5, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    ba

## Test iterating through loaded data, pulling out some info

In [21]:
'''   # not needed
for next_file in all_drummer_data.keys():
  perf_data = all_drummer_data[next_file]
  print('data: {}'.format(perf_data.file_wrapper))
  display(perf_data.stats_df)
'''

"   # not needed\nfor next_file in all_drummer_data.keys():\n  perf_data = all_drummer_data[next_file]\n  print('data: {}'.format(perf_data.file_wrapper))\n  display(perf_data.stats_df)\n"

# Filter/ merge, one-hot/ dummies encoding

Count available files, show names...

In [22]:
print('total files: {}'.format(len(all_drummer_data.keys())))
for next_file in all_drummer_data.keys():
  print('data for: {}'.format(all_drummer_data[next_file].file_wrapper.my_file_name))

total files: 20
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/3_soul-groove3_86_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/4_soul-groove4_80_beat_4-4.mid
data for: /content/drive/My Drive/groove-v1.

Now extract just first song style for all 4 drummers..

In [23]:
style_filter = "1_funk-groove1"  # first style to experiment with

for next_file in all_drummer_data.keys():
  perf_data = all_drummer_data[next_file]
  file_name = perf_data.file_wrapper.my_file_name
  if style_filter in file_name:
    print('data for: {}'.format(file_name))

    # quick review head of df
    display(perf_data.file_df.head())

    # pull out notes in 1st bar
    display(perf_data.file_df.xs(16, level='bar_number')) 

    # pull out notes in 1st bar, 1st beat
    display(perf_data.file_df.xs((1,1), level=['bar_number', 'bar_beat_number'])) 

    # pull out certain notes
    display(perf_data.file_df.xs(36, level='note')) 



data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid


track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number note                                              
1          1               49             0:6            3    49        60   
                           51             0:7            7    51        55   
                           36             0:8           10    36        64   
           3               36            0:14          211    36        57   
                           51            0:16          238    51        31   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          1               49              3           0           1   
                           51              7           0           1   
                           36             10           0           1   
           3               36            -29         240           1   
                           51             -2         240           1   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          1               49                  1  
                           51                  1  
                           36                  1  
           3               36                  3  
                           51                  3

track_msg_num  total_ticks  note  velocity  beat_offset  \
bar_beat_number note                                                           
1               51          0:1225        28813    51        46           13   
2               38          0:1226        28882    38        32          -38   
3               36          0:1234        29025    36        49          -15   
                51          0:1235        29029    51        51          -11   
4               36          0:1240        29143    36        54          -17   
                51          0:1241        29152    51        45           -8   
5               38          0:1248        29270    38       104          -10   
                51          0:1249        29281    51        59            1   
7               51          0:1257        29512    51        46           -8   
8               38          0:1261        29626    38        80          -14   
9               51          0:1267        29748    51        30          -12   
10              38          0:1270        29856    38        37          -24   
11              49          0:1275        29967    49        84          -33   
                36          0:1278        29978    36       127          -22   
13              51          0:1287        30231    51        33           -9   
15              38          0:1290        30448    38       116          -32   
                51          0:1291        30452    51        74          -28   

                     beat_center  bar_number  bar_beat_number  
bar_beat_number note                                           
1               51         28800          16                1  
2               38         28920          16                2  
3               36         29040          16                3  
                51         29040          16                3  
4               36         29160          16                4  
                51         29160          16                4  
5               38         29280          16                5  
                51         29280          16                5  
7               51         29520          16                7  
8               38         29640          16                8  
9               51         29760          16                9  
10              38         29880          16               10  
11              49         30000          16               11  
                36         30000          16               11  
13              51         30240          16               13  
15              38         30480          16               15  
                51         30480          16               15

,track_msg_num,total_ticks,note,velocity,beat_offset,beat_center,bar_number,bar_beat_number
note,,,,,,,,
49,0:6,3,49,60,3,0,1,1
51,0:7,7,51,55,7,0,1,1
36,0:8,10,36,64,10,0,1,1


track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number                                              
1          1                         0:8           10    36        64   
           3                        0:14          211    36        57   
           11                       0:51         1163    36        59   
2          1                        0:83         1915    36        55   
           3                        0:92         2150    36        62   
           11                      0:128         3074    36        47   
           12                      0:136         3219    36        54   
3          1                       0:166         3836    36        57   
           3                       0:178         4072    36        62   
           11                      0:216         5038    36        66   
4          3                       0:254         5975    36        45   
           4                       0:259         6100    36        48   
           11                      0:296         6924    36        71   
5          1                       0:326         7657    36        50   
           3                       0:338         7898    36        63   
           11                      0:374         8835    36        49   
           12                      0:380         8958    36        48   
6          1                       0:414         9599    36        55   
           3                       0:424         9827    36        63   
           11                      0:462        10774    36        46   
           12                      0:469        10914    36        59   
7          1                       0:498        11530    36        62   
           3                       0:508        11766    36        59   
           11                      0:547        12726    36        73   
8          3                       0:585        13673    36        42   
           4                       0:592        13805    36        55   
           11                      0:625        14614    36        87   
9          1                       0:656        15335    36        60   
           3                       0:668        15575    36        60   
           11                      0:704        16506    36        47   
10         1                       0:739        17256    36        64   
           3                       0:752        17523    36       125   
           11                      0:790        18467    36        51   
           12                      0:797        18592    36        65   
11         1                       0:827        19185    36        67   
           3                       0:838        19442    36        65   
           11                      0:874        20389    36        71   
12         3                       0:913        21356    36        50   
           4                       0:919        21480    36        57   
           14                      0:970        22686    36        52   
13         1                       0:985        23032    36        69   
           3                       0:998        23281    36        60   
           11                     0:1033        24197    36        52   
           12                     0:1039        24343    36        60   
14         1                      0:1070        24952    36        61   
           3                      0:1079        25190    36        61   
           11                     0:1115        26126    36        46   
           12                     0:1122        26270    36        53   
15         1                      0:1152        26862    36        62   
           3                      0:1161        27107    36        61   
           11                     0:1198        28081    36        76   
16         3                      0:1234        29025    36        49   
           4                      0:1240        29143    36        54   
           1

...

data for: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/1_funk-groove1_138_beat_4-4.mid


track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number note                                              
1          1               36             0:6            0    36        67   
                           42             0:8            0    42        40   
           3               36            0:11          210    36        64   
                           42            0:13          228    42        37   
           5               42            0:17          468    42        83   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          1               36              0           0           1   
                           42              0           0           1   
           3               36            -30         240           1   
                           42            -12         240           1   
           5               42            -12         480           1   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          1               36                  1  
                           42                  1  
           3               36                  3  
                           42                  3  
           5               42                  5

track_msg_num  total_ticks  note  velocity  beat_offset  \
bar_beat_number note                                                           
1               42           0:686        28790    42        41          -10   
2               38           0:688        28884    38        57          -36   
3               36           0:691        29004    36        50          -36   
                42           0:693        29007    42        56          -33   
4               36           0:696        29124    36        46          -36   
5               42           0:699        29262    42       125          -18   
                38           0:700        29262    38       127          -18   
7               42           0:708        29515    42        45           -5   
8               38           0:710        29632    38       111           -8   
9               42           0:713        29753    42        67           -7   
10              38           0:715        29866    38        73          -14   
11              36           0:717        29990    36        76          -10   
                42           0:719        29992    42       116           -8   
13              42           0:723        30242    42        31            2   
15              38           0:725        30476    38       127           -4   

                     beat_center  bar_number  bar_beat_number  
bar_beat_number note                                           
1               42         28800          16                1  
2               38         28920          16                2  
3               36         29040          16                3  
                42         29040          16                3  
4               36         29160          16                4  
5               42         29280          16                5  
                38         29280          16                5  
7               42         29520          16                7  
8               38         29640          16                8  
9               42         29760          16                9  
10              38         29880          16               10  
11              36         30000          16               11  
                42         30000          16               11  
13              42         30240          16               13  
15              38         30480          16               15

,track_msg_num,total_ticks,note,velocity,beat_offset,beat_center,bar_number,bar_beat_number
note,,,,,,,,
36,0:6,0,36,67,0,0,1,1
42,0:8,0,42,40,0,0,1,1


track_msg_num  total_ticks  note  velocity  \
bar_number bar_beat_number                                              
1          1                         0:6            0    36        67   
           3                        0:11          210    36        64   
           11                       0:31         1194    36        52   
           12                       0:34         1301    36        47   
2          1                        0:47         1939    36        64   
           3                        0:55         2185    36        64   
           11                       0:77         3122    36        48   
           12                       0:80         3235    36        43   
3          1                        0:94         3859    36        62   
           3                       0:100         4092    36        67   
           11                      0:121         5032    36        67   
4          3                       0:140         5968    36        49   
           4                       0:146         6097    36        45   
           11                      0:169         6947    36        66   
5          1                       0:182         7691    36        71   
           3                       0:187         7924    36        73   
           11                      0:209         8872    36        54   
           12                      0:212         8992    36        47   
6          1                       0:227         9622    36        69   
           3                       0:233         9858    36        67   
           11                      0:257        10805    36        51   
           12                      0:260        10916    36        46   
7          1                       0:274        11528    36        69   
           3                       0:279        11756    36        70   
           11                      0:303        12706    36        74   
8          1                       0:319        13453    36        65   
           3                       0:325        13681    36        49   
           11                      0:349        14640    36        67   
9          1                       0:362        15363    36        73   
           3                       0:367        15593    36        72   
           11                      0:391        16555    36        54   
           12                      0:395        16671    36        49   
10         1                       0:410        17287    36        72   
           3                       0:415        17518    36        70   
           11                      0:439        18482    36        47   
           12                      0:442        18600    36        49   
11         1                       0:457        19192    36        67   
           3                       0:463        19427    36        63   
           11                      0:487        20395    36        80   
12         3                       0:509        21349    36        53   
           4                       0:514        21482    36        52   
           11                      0:533        22340    36        75   
13         1                       0:546        23064    36        72   
           3                       0:551        23287    36        67   
           11                      0:577        24240    36        50   
           12                      0:580        24356    36        47   
14         1                       0:596        24965    36        68   
           3                       0:601        25202    36        63   
           11                      0:623        26168    36        50   
           12                      0:628        26284    36        49   
15         1                       0:641        26888    36        74   
           3                       0:647        27119    36        61   
           11                      0:669        28066    36        78   
16         3

## One file: filter features, drop fancy index

In [24]:

# experimenting on a single file here ...
key = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'

test_data = all_drummer_data[key]



In [25]:
type(test_data)
test_data.file_df.info()

__main__.PerformanceData

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 264 entries, (1, 1, 49) to (16, 15, 51)
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   track_msg_num    264 non-null    string  
 1   total_ticks      264 non-null    int64   
 2   note             264 non-null    int64   
 3   velocity         264 non-null    int64   
 4   beat_offset      264 non-null    int64   
 5   beat_center      264 non-null    category
 6   bar_number       264 non-null    int64   
 7   bar_beat_number  264 non-null    int64   
dtypes: category(1), int64(6), string(1)
memory usage: 17.7 KB


In [38]:

df1 = test_data.file_df.copy()

# create & set vanilla index
seq_index = pd.Series(range(1, len(df1)+1), name='NewIndex')
#df1.set_index(seq_index, inplace=True)
df1.set_index(seq_index, inplace=True, append=True)

# drop what we don't need
df1.drop(columns=['track_msg_num', 'total_ticks', 'beat_center', 'bar_number'], inplace=True)


>> LOADING custom module, when: 2020-07-31 19:35:55.839047, module name: data_prep
>> LOADING custom module, when: 2020-07-31 19:35:56.234151, module name: stats_n_features


In [39]:

# review progress
df1.info()
df1.head(50)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 264 entries, (1, 1, 49, 1) to (16, 15, 51, 264)
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   note             264 non-null    int64
 1   velocity         264 non-null    int64
 2   beat_offset      264 non-null    int64
 3   bar_beat_number  264 non-null    int64
dtypes: int64(4)
memory usage: 12.1 KB


note  velocity  beat_offset  \
bar_number bar_beat_number note NewIndex                                
1          1               49   1           49        60            3   
                           51   2           51        55            7   
                           36   3           36        64           10   
           3               36   4           36        57          -29   
                           51   5           51        31           -2   
           5               51   6           51        54          -13   
                           38   7           38        93           -5   
           7               51   8           51        42          -16   
           8               38   9           38        33          -12   
           10              38   10          38        33          -14   
           11              36   11          36        59          -37   
                           51   12          51        42          -32   
           13              51   13          51        64          -18   
                           38   14          38       114            7   
           15              51   15          51        44           -1   
           16              38   16          38        30           -3   
2          1               51   17          51        42           -7   
                           36   18          36        55           -5   
           3               36   19          36        62          -10   
                           51   20          51        41            3   
           5               38   21          38        94            4   
                           51   22          51        51            5   
           7               51   23          51        45            7   
           8               38   24          38        32            7   
           9               51   25          51        43            2   
           10              38   26          38        24           -5   
           11              36   27          36        47          -46   
                           51   28          51        36            2   
           12              36   29          36        54          -21   
           13              38   30          38        89          -13   
                           51   31          51        54            0   
           15              51   32          51        39            7   
3          1               36   33          36        57           -4   
                           51   34          51        45           -3   
           3               36   35          36        62           -8   
                           51   36          51        45            0   
           5               38   37          38        88           -3   
                           51   38          51        50            1   
           7               51   39          51        36            4   
           8               38   40          38        86          -12   
           9               51   41          51        49            6   
           10              38   42          38        27          -15   
           11              36   43          36        66           -2   
                           51   44          51        47            0   
           13              51   45          51        39          -11   
           15              51   46          51        51          -13   
                           38   47          38        94           -2   
4          1               51   48          51        43          -12   
           2               38   49          38        38          -35   
           3               51   50          51        41          -29   

                                          bar_beat_number  
bar_number bar_beat_number note NewIndex                   
1          1               49   1                       1  
                           51   2                       1  
     

## One file: review and filter instruments

Take a peek at the instruments we have...

In [28]:
# list of instrument numbers
print('instruments: {}'.format(df1['note'].unique()))

# english text what they are
dp.MidiTools.getInstruments(df1['note'].unique())

# frequency count
df1.groupby('note')['note'].count()

instruments: [49 51 36 38 43]


['Crash Cymbal 1 (49)',
 'Ride Cymbal 1 (51)',
 'Bass Drum 1 (36)',
 'Acoustic Snare (38)',
 'High Floor Tom (43)']

note
36     54
38     79
43      1
49      4
51    126
Name: note, dtype: int64

Filter to kick, snare and hats/ ride...

In [29]:
df2 = df1[df1.note.isin([36, 38, 51])]
df2.head()
# frequency count
df2.groupby('note')['note'].count()

,note,velocity,beat_offset,bar_beat_number
NewIndex,,,,
2,51,55,7,1
3,36,64,10,1
4,36,57,-29,3
5,51,31,-2,3
6,51,54,-13,5


note
36     54
38     79
51    126
Name: note, dtype: int64

Replace with 36=1 (kick), 38=2 (snare), 51=3 (hh/ride)

For more info, see...
* https://www.kite.com/python/answers/how-to-replace-column-values-in-a-pandas-dataframe-in-python
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

In [30]:
df3 = df2.copy()
#df3.note.replace({36: 1, 38: 2, 51:3 }, inplace=True)
df3.note.replace({36: 'kick', 38: 'snare', 51:'hh' }, inplace=True)

IMPORTANT: I *think* the next cell output shows the shape of the data I need to create BoxPlots???

In [31]:
df3.head(20)
# frequency count
df3.groupby('note').count()

,note,velocity,beat_offset,bar_beat_number
NewIndex,,,,
2,hh,55,7,1
3,kick,64,10,1
4,kick,57,-29,3
5,hh,31,-2,3
6,hh,54,-13,5
7,snare,93,-5,5
8,hh,42,-16,7
9,snare,33,-12,8
10,snare,33,-14,10


,velocity,beat_offset,bar_beat_number
note,,,
hh,126,126,126
kick,54,54,54
snare,79,79,79


## One file: one-hot, get_dummies

One-Hot, get_dummies stuff, a la..
*  https://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example
* https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding
* alternative, see Albon book, pg82, sklearn.preprocessing.LabelBinarizer

In [32]:
# testing ...

# one-hot the column specifying kick/ snare/ hh
pd.get_dummies(df3.note, prefix='note', drop_first=True).head(10) 

# one-hot the column specifying metric position in the bar
pd.get_dummies(df3.bar_beat_number, prefix='beat', drop_first=True).head(10)

,note_kick,note_snare
NewIndex,,
2,0,0
3,1,0
4,1,0
5,0,0
6,0,0
7,0,1
8,0,0
9,0,1
10,0,1


,beat_2,beat_3,beat_4,beat_5,beat_7,beat_8,beat_9,beat_10,beat_11,beat_12,beat_13,beat_14,beat_15,beat_16
NewIndex,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0,0,0,0


Now do it for real, append the new one-hot cols, remove the cols that made them...

In [33]:
df4 = df3.copy()

# join the new columns with  original dataframe
df4 = pd.concat([df4,pd.get_dummies(df4.note, prefix='note', drop_first=True)],axis=1)
df4.drop('note',axis=1, inplace=True)  # drop original 'note' col

df4 = pd.concat([df4,pd.get_dummies(df4.bar_beat_number, prefix='beat', drop_first=True)],axis=1)
df4.drop('bar_beat_number',axis=1, inplace=True)  # drop original 'note' col

df4.head()

,velocity,beat_offset,note_kick,note_snare,beat_2,beat_3,beat_4,beat_5,beat_7,beat_8,beat_9,beat_10,beat_11,beat_12,beat_13,beat_14,beat_15,beat_16
NewIndex,,,,,,,,,,,,,,,,,,
2,55,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,64,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,57,-29,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,31,-2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,54,-13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


# Todo: normalization

Normalize - don't know distribution, or know it's not guassian, algorithm doesn't make assumptions about distribution

Standardization - assumes data is Guassian/ bell shaped, and when algorithms assumes Guassian.

Final step to prep data for KNN, I need to normalize the columns: 'velocity', 'beat_offset' ..

In [34]:
# take a look at what we have so far...
df4.describe()

,velocity,beat_offset,note_kick,note_snare,beat_2,beat_3,beat_4,beat_5,beat_7,beat_8,beat_9,beat_10,beat_11,beat_12,beat_13,beat_14,beat_15,beat_16
count,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000
mean,59.023166,-8.382239,0.208494,0.305019,0.019305,0.123552,0.027027,0.123552,0.061776,0.061776,0.057915,0.057915,0.104247,0.030888,0.092664,0.003861,0.092664,0.003861
std,23.861508,16.705284,0.407018,0.461307,0.137861,0.329707,0.162476,0.329707,0.241214,0.241214,0.234035,0.234035,0.306172,0.173349,0.290523,0.062137,0.290523,0.062137
min,8.000000,-58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,44.000000,-17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,52.000000,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,65.500000,1.500000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,127.000000,55.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Todo: KNN experiments

Just a wee reminder, some handy links...

Tutorial on manipulating MultiIndexes here ...
* https://github.com/ZaxR/pandas_multiindex_tutorial/blob/master/Pandas%20MultiIndex%20Tutorial.ipynb


Example using df.xs to extract info...
* display(multi_col_lvl_df.xs(('2018-07-10', 'Booooze', 'Wine'), level=['Date', 'Department', 'Category']).head(10))


One Hot Encoding info..
* https://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example



Dataframe.xs (cross section) ...
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html




```
For KNN, data needs to be in format ..
- Observation X = Class Y

Suggested data format
- X = [instrument, metric position, offset, velocity]
- Y = best guess at which drummer it is.

Details..
- instrument = kick, snare, hh/ ride (categorical, order not important = nominal categorical)
- metric position = 1 to 16  (numeric, order not important = nominal categorical)
- offset = -60 to +60  (offset range is # ticks per beat, +/- half of that)
- velocity = 0 to 127 (MIDI volume range is 1 to 127)
```


